In [24]:
# Cell 0: Install required libraries
!pip install -q transformers datasets evaluate peft bitsandbytes accelerate gradio sentence-transformers faiss-cpu kaggle scikit-learn

# Disable wandb by default (prevents API prompt)
import os
os.environ["WANDB_DISABLED"] = "true"
print("Installed packages and disabled wandb.")


Installed packages and disabled wandb.


In [25]:
# ============================================================
# 🧠 Download Mental Health Datasets from Kaggle (Safe Setup)
# ============================================================

# --- Step 1: Upload your kaggle.json (contains your Kaggle API credentials) ---
# 👉 Only run this once per Colab session
from google.colab import files
print("📂 Please upload your kaggle.json file (download it from your Kaggle account).")
files.upload()

# --- Step 2: Move kaggle.json to the correct directory ---
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# --- Step 3: Create a directory for Kaggle datasets ---
!mkdir -p /content/kaggle_data

# --- Step 4: Download and unzip datasets ---
# You can comment out any dataset you don’t want to download
print("⬇️ Downloading datasets... please wait!")

!kaggle datasets download -d suchintikasarkar/sentiment-analysis-for-mental-health -p /content/kaggle_data --unzip
!kaggle datasets download -d elvis23/mental-health-conversational-data -p /content/kaggle_data --unzip
!kaggle datasets download -d nikhileswarkomati/suicide-watch -p /content/kaggle_data --unzip

# --- Step 5: Check downloaded files ---
print("📁 Downloaded files:")
!ls -lh /content/kaggle_data


📂 Please upload your kaggle.json file (download it from your Kaggle account).


Saving kaggle.json to kaggle.json
⬇️ Downloading datasets... please wait!
Dataset URL: https://www.kaggle.com/datasets/suchintikasarkar/sentiment-analysis-for-mental-health
License(s): DbCL-1.0
  0% 0.00/11.1M [00:00<?, ?B/s]
100% 11.1M/11.1M [00:00<00:00, 1.45GB/s]
Dataset URL: https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
License(s): copyright-authors
  0% 0.00/11.8k [00:00<?, ?B/s]
100% 11.8k/11.8k [00:00<00:00, 56.7MB/s]
Dataset URL: https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch
License(s): CC-BY-SA-4.0
  0% 0.00/60.6M [00:00<?, ?B/s]
100% 60.6M/60.6M [00:00<00:00, 1.73GB/s]
📁 Downloaded files:
total 190M
-rw-r--r-- 1 root root  31M Oct 22 11:13 'Combined Data.csv'
-rw-r--r-- 1 root root  39K Oct 22 11:13  intents.json
-rw-r--r-- 1 root root 160M Oct 22 11:13  Suicide_Detection.csv


In [26]:
# Cell 2: Load the three dataset files into pandas DataFrames
import pandas as pd, json, os, glob

DATA_DIR = "/content/kaggle_data"
print("Files in dataset folder:", os.listdir(DATA_DIR))

# 1) Sentiment dataset — Combined Data.csv (this file came from earlier dataset)
sentiment_file = None
for f in os.listdir(DATA_DIR):
    if "combined" in f.lower() or "combined data" in f.lower() or "sentiment" in f.lower():
        sentiment_file = os.path.join(DATA_DIR, f)
        break
if sentiment_file:
    df_sent = pd.read_csv(sentiment_file, encoding='utf-8', low_memory=False)
    print("Loaded sentiment file:", sentiment_file, "shape:", df_sent.shape)
else:
    print("No sentiment CSV found automatically. List files and pick manually:")
    print(os.listdir(DATA_DIR))

# 2) Conversational dataset — intents.json
convo_file = None
for f in os.listdir(DATA_DIR):
    if f.lower().endswith(".json"):
        convo_file = os.path.join(DATA_DIR, f)
        break

if convo_file:
    with open(convo_file, 'r', encoding='utf-8') as fh:
        j = json.load(fh)
    # Convert intents.json -> rows (if structure matches)
    rows = []
    if isinstance(j, dict) and ("intents" in j):
        for intent in j["intents"]:
            patterns = intent.get("patterns", [])
            responses = intent.get("responses", [])
            tag = intent.get("tag", "neutral")
            for p in patterns:
                for r in responses:
                    rows.append({"user_text": p, "assistant_text": r, "sentiment": tag})
    else:
        # try to parse a generic list of objects
        if isinstance(j, list):
            for item in j:
                rows.append(item)
    df_convo = pd.DataFrame(rows)
    print("Loaded conversational JSON:", convo_file, "-> rows:", df_convo.shape)
else:
    print("No conversational json found.")

# 3) Suicide / crisis dataset — CSV
suicide_file = None
for f in os.listdir(DATA_DIR):
    if "suicide" in f.lower() or "suicide_detection" in f.lower():
        suicide_file = os.path.join(DATA_DIR, f)
        break
if suicide_file:
    try:
        df_suicide = pd.read_csv(suicide_file, encoding='utf-8', low_memory=False)
    except:
        df_suicide = pd.read_csv(suicide_file, encoding='latin1', low_memory=False)
    print("Loaded suicide file:", suicide_file, "shape:", df_suicide.shape)
else:
    print("No suicide-watch CSV found.")

# Quick preview (if exists)
for name, df in [("df_sent", globals().get("df_sent", None)), ("df_convo", globals().get("df_convo", None)), ("df_suicide", globals().get("df_suicide", None))]:
    if df is not None:
        print(f"\n--- {name} preview ---")
        display(df.head())


Files in dataset folder: ['intents.json', 'Combined Data.csv', 'Suicide_Detection.csv']
Loaded sentiment file: /content/kaggle_data/Combined Data.csv shape: (53043, 3)
Loaded conversational JSON: /content/kaggle_data/intents.json -> rows: (661, 3)
Loaded suicide file: /content/kaggle_data/Suicide_Detection.csv shape: (232074, 3)

--- df_sent preview ---


,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety



--- df_convo preview ---


,user_text,assistant_text,sentiment
0,Hi,Hello there. Tell me how are you feeling today?,greeting
1,Hi,Hi there. What brings you here today?,greeting
2,Hi,Hi there. How are you feeling today?,greeting
3,Hi,Great to see you. How do you feel currently?,greeting
4,Hi,Hello there. Glad to see you're back. What's g...,greeting



--- df_suicide preview ---


,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [27]:
# Cell 3: Preprocess and standardize datasets
import numpy as np

# 1) Clean sentiment dataset -> df_sent_clean
if 'df_sent' in globals():
    # Find likely text & label columns
    sent_df = df_sent.copy()
    sent_df.columns = [c.strip().lower() for c in sent_df.columns]
    # Heuristics to pick columns
    text_col = next((c for c in sent_df.columns if "text" in c or "statement" in c or "sentence" in c or "message" in c), None)
    label_col = next((c for c in sent_df.columns if "label" in c or "sentiment" in c or "status" in c), None)
    if text_col is None:
        text_col = sent_df.columns[0]
    if label_col is None:
        label_col = sent_df.columns[-1]
    df_sent_clean = sent_df[[text_col, label_col]].rename(columns={text_col: "text", label_col: "sentiment"}).dropna().reset_index(drop=True)
    df_sent_clean['sentiment'] = df_sent_clean['sentiment'].astype(str).str.lower().str.strip()
    print("df_sent_clean shape:", df_sent_clean.shape)
else:
    df_sent_clean = pd.DataFrame(columns=["text","sentiment"])
    print("No sentiment file available.")

# Normalize simple label mapping if numeric
def normalize_sentiment_label(s):
    s = str(s).lower().strip()
    if s in ["0","neg","negative","sad","sadness"]:
        return "negative"
    if s in ["1","neu","neutral","none","other"]:
        return "neutral"
    if s in ["2","pos","positive","happy","joy"]:
        return "positive"
    return s

if not df_sent_clean.empty:
    df_sent_clean['sentiment'] = df_sent_clean['sentiment'].apply(normalize_sentiment_label)

# 2) Conversational pairs -> df_convo_pairs
if 'df_convo' in globals():
    convo = df_convo.copy()
    # ensure columns exist
    if 'user_text' not in convo.columns:
        # try first two columns
        cols = convo.columns.tolist()
        if len(cols) >= 2:
            convo = convo.rename(columns={cols[0]:"user_text", cols[1]:"assistant_text"})
    if 'assistant_text' not in convo.columns:
        convo['assistant_text'] = "I'm sorry you're feeling this way. Can you tell me more?"
    if 'sentiment' not in convo.columns:
        convo['sentiment'] = "neutral"
    df_convo_pairs = convo[['user_text','assistant_text','sentiment']].dropna().reset_index(drop=True)
    df_convo_pairs['sentiment'] = df_convo_pairs['sentiment'].astype(str).str.lower().apply(normalize_sentiment_label)
    print("df_convo_pairs shape:", df_convo_pairs.shape)
else:
    df_convo_pairs = pd.DataFrame(columns=["user_text","assistant_text","sentiment"])
    print("No conversational pairs available.")

# 3) Suicide dataset -> df_suicide_clean (for crisis examples)
if 'df_suicide' in globals():
    s = df_suicide.copy()
    # Try to find a text column
    s_cols = [c for c in s.columns if "text" in c.lower() or "post" in c.lower() or "body" in c.lower() or "comment" in c.lower()]
    text_col = s_cols[0] if s_cols else s.columns[0]
    df_suicide_clean = pd.DataFrame({"text": s[text_col].astype(str).fillna("")})
    df_suicide_clean = df_suicide_clean[df_suicide_clean['text'].str.strip() != ""].reset_index(drop=True)
    print("df_suicide_clean shape:", df_suicide_clean.shape)
else:
    df_suicide_clean = pd.DataFrame(columns=["text"])
    print("No suicide dataset available.")

# Quick stats
print("\nCounts:")
print("Sentiment rows:", len(df_sent_clean))
print("Convo pairs rows:", len(df_convo_pairs))
print("Suicide rows:", len(df_suicide_clean))


df_sent_clean shape: (52681, 2)
df_convo_pairs shape: (661, 3)
df_suicide_clean shape: (232074, 1)

Counts:
Sentiment rows: 52681
Convo pairs rows: 661
Suicide rows: 232074


In [28]:
# ======================================================
# 🧠 Cell 4: Train Sentiment Classifier (DistilBERT)
# ======================================================
import numpy as np
import pandas as pd
from datasets import Dataset
from evaluate import load as load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# --- Version check for transformers ---
import transformers
transformers_version = transformers.__version__
print("Transformers version:", transformers_version)

# --- Prepare data ---
if df_sent_clean.shape[0] < 2000 and not df_convo_pairs.empty:
    add = df_convo_pairs[['user_text']].rename(columns={'user_text': 'text'})
    add['sentiment'] = "neutral"
    df_for_clf = pd.concat([df_sent_clean, add], ignore_index=True).drop_duplicates().reset_index(drop=True)
else:
    df_for_clf = df_sent_clean.copy()

if df_for_clf.empty:
    raise RuntimeError("No data found for classifier training. Ensure df_sent_clean or df_convo_pairs exist.")

# --- Label mapping ---
labels = sorted(df_for_clf['sentiment'].unique())
label2id = {l: i for i, l in enumerate(labels)}
df_for_clf['label_id'] = df_for_clf['sentiment'].map(label2id)

# --- Convert to HF Dataset ---
ds = Dataset.from_pandas(df_for_clf[['text', 'label_id']])
ds = ds.train_test_split(test_size=0.1, seed=42)

# --- Tokenization ---
model_name = "distilbert-base-uncased"
tokenizer_clf = AutoTokenizer.from_pretrained(model_name)

def tokenize_clf(batch):
    return tokenizer_clf(batch["text"], truncation=True, padding="max_length", max_length=128)

ds = ds.map(tokenize_clf, batched=True, remove_columns=["text"])
ds = ds.rename_column("label_id", "labels")
ds.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

# --- Model ---
model_clf = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels))

# --- Training arguments (version-safe) ---
try:
    training_args = TrainingArguments(
        output_dir="./clf_out",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=2,
        logging_steps=50,
        eval_strategy="epoch",   # ✅ newer transformers
        save_strategy="epoch",
        fp16=True,
        report_to=[]  # disables logging
    )
except TypeError:
    training_args = TrainingArguments(
        output_dir="./clf_out",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=2,
        logging_steps=50,
        evaluation_strategy="epoch",  # ✅ older transformers
        save_strategy="epoch",
        fp16=True,
        report_to=[]
    )

# --- Metrics ---
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels_eval = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": (preds == labels_eval).mean()}

# --- Trainer ---
trainer = Trainer(
    model=model_clf,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer_clf,
    compute_metrics=compute_metrics
)

# --- Train and save ---
print("🚀 Starting classifier training...")
trainer.train()
trainer.save_model("./sentiment_classifier")
tokenizer_clf.save_pretrained("./sentiment_classifier")
print("✅ Saved sentiment classifier. Label2id:", label2id)


Transformers version: 4.57.1


Map:   0%|          | 0/47412 [00:00<?, ? examples/s]

Map:   0%|          | 0/5269 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3380234685.py:89: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Starting classifier training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.445100,0.450273,0.811160
2,0.340900,0.437651,0.826912


✅ Saved sentiment classifier. Label2id: {'anxiety': 0, 'bipolar': 1, 'depression': 2, 'normal': 3, 'personality disorder': 4, 'stress': 5, 'suicidal': 6}


In [29]:
# Cell 5: Tag conversations and build generator dataset
from transformers import pipeline
import torch

# Load classifier pipeline
clf_pipe = pipeline("text-classification", model="./sentiment_classifier", tokenizer="./sentiment_classifier", device=0 if torch.cuda.is_available() else -1)

# Tag df_convo_pairs user_text (limit to N to save time)
if df_convo_pairs.empty:
    raise RuntimeError("No conversation pairs found to build generator dataset. Check earlier cells.")

sample_size = min(8000, len(df_convo_pairs))
df_sample = df_convo_pairs.sample(n=sample_size, random_state=42).reset_index(drop=True)

# batch classify
texts = df_sample['user_text'].astype(str).tolist()
batch_size = 32
pred_labels = []
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    preds = clf_pipe(batch)
    for p in preds:
        # pipeline returns labels like 'LABEL_0' or actual label string depending on tokenizer; handle both
        lab = p.get('label')
        if lab.startswith("LABEL_"):
            idx = int(lab.split("_")[-1])
            inv = {v:k for k,v in label2id.items()}
            pred_labels.append(inv.get(idx,"neutral"))
        else:
            pred_labels.append(lab.lower())

df_sample['pred_sentiment'] = pred_labels

# Add some crisis examples from suicide dataset and mark sentiment 'crisis'
crisis_texts = []
if not df_suicide_clean.empty:
    crisis_texts = df_suicide_clean['text'].astype(str).sample(n=min(500, len(df_suicide_clean)), random_state=42).tolist()

# Build generator pairs: input = "SENTIMENT: <sent> | USER: <user_text>"
pairs = []
for _, row in df_sample.iterrows():
    inp = f"SENTIMENT: {row['pred_sentiment'].upper()} | USER: {row['user_text']}"
    tgt = row['assistant_text']
    pairs.append({"input_text": inp, "target_text": tgt})

# Add crisis rows with empty target (we will handle as template at runtime)
for t in crisis_texts:
    inp = f"SENTIMENT: CRISIS | USER: {t}"
    tgt = "CRISIS_TEMPLATE"  # placeholder, model should not be used for crisis replies
    pairs.append({"input_text": inp, "target_text": tgt})

df_gen = pd.DataFrame(pairs).drop_duplicates().reset_index(drop=True)
print("Generator dataset size:", len(df_gen))
display(df_gen.sample(n=min(5,len(df_gen))))


Device set to use cuda:0


Generator dataset size: 1161


,input_text,target_text
538,SENTIMENT: NORMAL | USER: What is a therapist?,A therapist is a broad designation that refers...
673,"SENTIMENT: CRISIS | USER: suicidal ideation, f...",CRISIS_TEMPLATE
578,SENTIMENT: NORMAL | USER: Bonjour,Hello there. Tell me how are you feeling today?
242,SENTIMENT: NORMAL | USER: boyfriend,Oh okay. Why don't you tell me more about it?
1063,SENTIMENT: CRISIS | USER: Please help. I don’t...,CRISIS_TEMPLATE


In [30]:
from transformers import Seq2SeqTrainingArguments

try:
    training_args = Seq2SeqTrainingArguments(
        output_dir="./t5_mindcare",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        predict_with_generate=True,
        fp16=True,
        num_train_epochs=2,
        logging_steps=50,
        save_strategy="epoch",
        eval_strategy="no",  # ✅ newer transformers
        learning_rate=3e-4,
        report_to=[]  # disables logging safely
    )
except TypeError:
    training_args = Seq2SeqTrainingArguments(
        output_dir="./t5_mindcare",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        predict_with_generate=True,
        fp16=True,
        num_train_epochs=2,
        logging_steps=50,
        save_strategy="epoch",
        evaluation_strategy="no",  # ✅ older transformers
        learning_rate=3e-4,
        report_to=[]
    )


In [31]:
# Cell 6: Train T5 Generator Model
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Convert dataframe to Hugging Face dataset
gen_ds = Dataset.from_pandas(df_gen).train_test_split(test_size=0.1, seed=42)

# Tokenize the dataset for T5
tokenizer = T5Tokenizer.from_pretrained("t5-small")
max_input = 128
max_target = 64

def tokenize_gen(batch):
    model_inputs = tokenizer(batch["input_text"], max_length=max_input, truncation=True, padding="max_length")
    labels = tokenizer(batch["target_text"], max_length=max_target, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

gen_ds = gen_ds.map(tokenize_gen, batched=True, remove_columns=["input_text", "target_text"])

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Use the training arguments defined in Cell 6
# training_args = Seq2SeqTrainingArguments(...) # defined in cell 6

# Initialize Trainer
trainer_gen = Seq2SeqTrainer(
    model=model,
    args=training_args, # Use args from Cell 6
    train_dataset=gen_ds["train"],
    eval_dataset=gen_ds["test"],
    tokenizer=tokenizer
)

# Train the model
print("🚀 Starting generator training...")
trainer_gen.train()

# Save the model
trainer_gen.save_model("./t5_mindcare")
tokenizer.save_pretrained("./t5_mindcare")
print("✅ Saved T5 generator model.")

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

/tmp/ipython-input-4200259846.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_gen = Seq2SeqTrainer(


🚀 Starting generator training...


Step,Training Loss
50,2.732000
100,0.477500
150,0.349100
200,0.368600
250,0.316800


✅ Saved T5 generator model.


In [33]:
# Cell 7: Crisis detector + Gradio demo
import re
from transformers import pipeline
import gradio as gr

# Load generator model & tokenizer
tokenizer = T5Tokenizer.from_pretrained("./t5_mindcare")
model = T5ForConditionalGeneration.from_pretrained("./t5_mindcare").to("cuda" if torch.cuda.is_available() else "cpu")

# Load sentiment classifier pipeline for runtime tagging
clf_pipe_runtime = pipeline("text-classification", model="./sentiment_classifier", tokenizer="./sentiment_classifier", device=0 if torch.cuda.is_available() else -1)

# Very conservative crisis keyword list (expand cautiously)
CRISIS_KEYWORDS = [
    "i want to die", "i want to kill myself", "suicide", "end my life", "hurt myself",
    "kill myself", "i'm going to kill myself", "i'm going to end my life", "cant go on", "cant take it"
]
def crisis_check(text):
    t = text.lower()
    for kw in CRISIS_KEYWORDS:
        if kw in t:
            return True
    if re.search(r"\bkill myself\b|\bend my life\b|\bwant to die\b|\bi can't go on\b|\bi cannot go on\b", t):
        return True
    return False

# Crisis response template (very conservative)
CRISIS_TEMPLATE = (
    "I’m really sorry you're feeling this way. I’m not able to provide emergency help. "
    "If you are in immediate danger, please call your local emergency number now. "
    "If you can, please reach out to a trusted person nearby or a local crisis hotline. "
    "Would you like resources for your country?"
)

# Generation function with sentiment conditioning and sampling
device = "cuda" if torch.cuda.is_available() else "cpu"
def generate_reply(user_text):
    # crisis check first
    if crisis_check(user_text):
        return CRISIS_TEMPLATE

    # get sentiment (runtime)
    try:
        pred = clf_pipe_runtime(user_text)[0]
        lab = pred.get('label')
        # if pipeline returns LABEL_0 style map to label2id
        if lab.startswith("LABEL_"):
            idx = int(lab.split("_")[-1])
            inv = {v:k for k,v in label2id.items()}
            sentiment = inv.get(idx, "neutral")
        else:
            sentiment = lab.lower()
    except Exception as e:
        sentiment = "neutral"

    prompt = f"SENTIMENT: {sentiment.upper()} | USER: {user_text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(
        **inputs,
        max_length=120,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        temperature=0.7,
        no_repeat_ngram_size=2,
        num_return_sequences=1
    )
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # If model learned crisis placeholder, protect: replace it with safe template
    if "CRISIS_TEMPLATE" in reply or reply.strip().upper().startswith("CRISIS"):
        return CRISIS_TEMPLATE
    # Final safety fallback
    if not reply.strip():
        return "Sorry, I couldn't generate a response. Would you like to try rephrasing or tell me more?"
    return reply

# Gradio interface
import html
def chat_interface(user_text):
    return generate_reply(user_text)

demo = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(lines=3, placeholder="Share how you're feeling..."),
    outputs="text",
    title="MindCare (demo)",
    description="Prototype mental-health support LLM (research/hobby demo). Not a replacement for professional help."
)
demo.launch()


Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1cd2e737bd236cfb54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
